In [1]:
import pandas as pd
import numpy as np

In [2]:
from log_progress import log_progress

In [3]:
import matplotlib
%matplotlib inline

In [4]:
import seaborn as sns

# Test Public

In [16]:
public_test_df = pd.read_csv('data/test-public.txt', sep='\t', index_col='Id')
public_test_df.head()

,Source,Sink
Id,,
1,2184483,1300190
2,3151356,1452193
3,1579396,193159
4,1406432,2481036
5,2389638,593017


In [6]:
source_set = set(public_test_df.Source.values)
sink_set = set(public_test_df.Sink.values)
len(sink_set), len(source_set), len(source_set.intersection(sink_set))

(1978, 2000, 30)

In [7]:
len(source_set.intersection(sink_set))

30

In [8]:
test_set = source_set.union(sink_set)

# Train data load

## Edges Dataframe

In [14]:
#edges_df = pd.read_csv()

## Nodes Dataframe

In [10]:
nodes_df = pd.read_csv('data/node_features.csv', index_col='node_id')
nodes_df.head()

/anaconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,is_parent_node,is_leaf_node,parents_count,children_count,children_links_counts,parents_links_counts
node_id,,,,,,
0,0,1,3,0,0,0
1,0,1,3,0,0,0
2,0,1,2,0,0,0
3,0,1,2,0,0,0
4,0,1,3,0,0,0


In [11]:
#nodes_df = nodes_df.iloc[:int(2e4)].copy()
nodes_df.shape

(4867136, 6)

In [12]:
nodes_df[(nodes_df.parents_count > 0) & (nodes_df.children_count > 0)].shape

(19570, 6)

In [13]:
nodes_df[nodes_df.parents_links_counts > 0].head()

,is_parent_node,is_leaf_node,parents_count,children_count,children_links_counts,parents_links_counts
node_id,,,,,,
1097,1,0,21,176,25197,34232
1122,1,0,109,1012,91833,50033
1152,1,0,94,2777,538818,877504
1170,1,0,138,452,70953,360778
1174,1,0,187,542,1066453,1097483


# Join data

In [17]:
public_test_df.head()

,Source,Sink
Id,,
1,2184483,1300190
2,3151356,1452193
3,1579396,193159
4,1406432,2481036
5,2389638,593017


In [18]:
nodes_df.head()

,is_parent_node,is_leaf_node,parents_count,children_count,children_links_counts,parents_links_counts
node_id,,,,,,
0,0,1,3,0,0,0
1,0,1,3,0,0,0
2,0,1,2,0,0,0
3,0,1,2,0,0,0
4,0,1,3,0,0,0


In [24]:
source_columns = ['parents_count', 'children_count', 'children_links_counts', 'parents_links_counts']
train_df = public_test_df.join(nodes_df[source_columns],
                    on='Source', rsuffix='source')

In [27]:
sink_columns = ['is_parent_node', 'is_leaf_node', 'parents_count',
                  'children_count', 'children_links_counts', 'parents_links_counts']
train_df = train_df.join(nodes_df[sink_columns], on='Sink', rsuffix='_sink')

In [29]:
train_df.head(100)

,Source,Sink,parents_count,children_count,children_links_counts,parents_links_counts,is_parent_node,is_leaf_node,parents_count_sink,children_count_sink,children_links_counts_sink,parents_links_counts_sink
Id,,,,,,,,,,,,
1,2184483,1300190,102,83,75149,238206,0,1,3,0,0,0
2,3151356,1452193,39,340,190693,84953,0,1,289,0,0,0
3,1579396,193159,13,208,389996,326756,0,1,2,0,0,0
4,1406432,2481036,16,84,31560,29915,1,0,24,14,292,1357
5,2389638,593017,30,267,146021,392713,1,0,165,58,42646,175340
6,228206,212805,14,9,1659,2939,1,0,894,297,640235,2109580
7,1237964,879115,4,15,6628,5626,0,1,482,0,0,0
8,3318124,1840575,26,93,6232,11096,0,1,26,0,0,0
9,4522929,1552625,46,422,134229,647766,0,1,63,0,0,0


In [56]:
mask = (train_df.children_count_sink > 0)# | (train_df.parents_count_sink > 100)

In [62]:
train_df['prediction'] = mask.astype(int)

In [63]:
train_df.set_index(['Source', 'Sink']).head()

,Source,Sink,parents_count,children_count,children_links_counts,parents_links_counts,is_parent_node,is_leaf_node,parents_count_sink,children_count_sink,children_links_counts_sink,parents_links_counts_sink,prediction
Id,,,,,,,,,,,,,
1,2184483,1300190,102,83,75149,238206,0,1,3,0,0,0,0
2,3151356,1452193,39,340,190693,84953,0,1,289,0,0,0,0
3,1579396,193159,13,208,389996,326756,0,1,2,0,0,0,0
4,1406432,2481036,16,84,31560,29915,1,0,24,14,292,1357,1
5,2389638,593017,30,267,146021,392713,1,0,165,58,42646,175340,1


In [64]:
nodes_шеdf.head(100)

,is_parent_node,is_leaf_node,parents_count,children_count,children_links_counts,parents_links_counts
node_id,,,,,,
0,0,1,3,0,0,0
1,0,1,3,0,0,0
2,0,1,2,0,0,0
3,0,1,2,0,0,0
4,0,1,3,0,0,0
5,0,1,2,0,0,0
6,0,1,1,0,0,0
7,0,1,1,0,0,0
8,0,1,1,0,0,0


# Predictions Submission

In [60]:
train_df['prediction'] = 1
#train_df['prediction'] = mask.astype(int)
train_df.loc[mask, 'prediction'] = 0
train_df[['prediction']].to_csv('prediction.csv')
train_df.prediction.sum()

1631

In [ ]:
mask = data.Source.isin(test_set) | data.Sink.isin(test_set)
data = data[mask]
mask.sum()

In [ ]:
data.head()

In [ ]:
data['is_train_set'] = 1
data['Id'] = range(2001, 2001 + data.shape[0], 1)

In [ ]:
public_test_df['is_train_set'] = 0

In [ ]:
df = pd.concat([public_test_df, data], axis=0)

In [ ]:
df.head()

In [ ]:
child_count = lambda x: node_child_counts[x] if x in node_child_counts.keys() else 0
df['source_childs_count'] = df.Source.apply(child_count).clip(upper=1000)
df['sink_childs_count'] = df.Sink.apply(child_count).clip(upper=1000)

In [ ]:
def intersect(vector):
    if vector['Sink'] not in node_child_sets.keys():
        return 0
    set1 = node_child_sets[vector['Source']]
    set2 = node_child_sets[vector['Sink']] 
    return len(set1.intersection(set2))
df['source_sink_child_intersect_count'] = df.apply(intersect, axis=1)

In [ ]:
df[df.source_childs_count > 0]

In [ ]:
sns.pairplot(df.ix[:4000, 3:], hue='is_train_set', size=4)

In [ ]:
df.Source.value_counts().head()

Intersection of source nodes Kaggge Test Set and Train Data Source Node Set

In [ ]:
len(source_set.intersection(set(data.source.values)))

In [ ]:
len(sink_set.intersection(set(data.sink.values)))

In [ ]:
public_test_df.head()

In [ ]:
df = data.source

In [ ]:
df.dropna()

In [ ]:
len(set(data.source.values).intersection(set(data.sink.values)))

In [ ]:
data.groupby('source')['sink'].count()

# Predictions

In [ ]:
predictions = pd.read_csv('data/sample.csv')
predictions['Prediction'] = 0

In [ ]:
predictions.to_csv('prediction.csv', index=False)

In [ ]:
df['prediction'] = (df['source_childs_count'] > 200).astype(int)
df[df.Id <= 2000][['Id', 'prediction']].to_csv('prediction.csv', index=False)